# Prediction from classifier

In [34]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import DataLoader
cudnn.benchmark = True
plt.ion()   # interactive mode
from utils.TestImagesDataset import TestImagesDataset
from tqdm import tqdm
CLASSES=14
device = torch.device("cuda:0" if torch.cuda.is_available() else  "cpu")

In [35]:
data_transforms = transforms.Compose([transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])

test_set = TestImagesDataset(data_transforms)
test_set.__len__()

184003

In [44]:
bs=256
dataset_size = test_set.__len__()
dataloader = DataLoader(dataset=test_set,
                          batch_size=bs,
                          shuffle=False,
                          num_workers=4)

In [45]:
EFFICIENTNET=True
model_ft=None

if EFFICIENTNET:
    model_ft = models.efficientnet_b0(pretrained=True)
    model_ft.classifier[1] = nn.Linear(in_features=1280, out_features=CLASSES)
    model_ft = model_ft.to(device)
    #reloading saved weights
    model_ft.load_state_dict(torch.load('./models/efficientnet_b0.pt'))
    model_ft.eval()
else: # RESNET
    model_ft = models.resnet18(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, CLASSES)
    model_ft = model_ft.to(device)

    #reloading saved weights
    model_ft.load_state_dict(torch.load('./models/resnet18.pt'))
    model_ft.eval()



with torch.no_grad():
    n_correct = 0
    n_samples = 0
    predictions=[]
    with tqdm(dataloader, unit="batch") as tepoch: 
        for images in tepoch:
            tepoch.set_description("Predicting...")
            images = images.to(device)
            outputs = model_ft(images)
            _, preds = torch.max(outputs, 1)
           
            predictions.append(preds)


Predicting...: 100%|███████████████████████████████████████████████████████| 719/719 [06:02<00:00,  1.99batch/s]


In [50]:
all_preds=torch.cat(predictions, dim=0).cpu().detach().numpy()

In [51]:
all_preds[0:10]

array([13, 13, 13, 13, 13, 13, 13, 13, 13, 13])

In [52]:
paths=[]
for i in range(71,126):
    paths.append('RALIHR_surgeon01_fps01_'+('0000'+str(i))[-4:])

for i in range(1,5):
    paths.append('RALIHR_surgeon02_fps01_'+('0000'+str(i))[-4:])

paths.append('RALIHR_surgeon03_fps01_0001')
vid2len={}
import os
ROOT='./e6691-bucket-images/'
index2data=[]
for vid in paths:
    vid2len[vid]=len(os.listdir(os.path.join(ROOT,vid)))
    for i in range (len(os.listdir(os.path.join(ROOT,vid)))):
        index2data.append(os.path.join(vid,'frame'+str(i)+'.jpg'))
        

In [53]:
import pandas as pd
df=pd.read_csv('./kaggle_template.csv')

In [78]:
predictions=all_preds
correct_length_seq=[]
for vid in vid2len:
    seq=predictions[:vid2len[vid]].tolist()
    predictions=predictions[vid2len[vid]:]
    rows=vid[14:16]+'-'+vid[23:27]+'-'
    seq_len=len(df[df['Id'].str.contains(rows)])
    if seq_len>len(seq):
        print('pad',seq_len-len(seq))
        seq.extend([seq[-1]] * (seq_len-len(seq)))
        
    else:
        print('nopad',seq_len-len(seq))
        seq=seq[:seq_len]
    correct_length_seq+=seq
len(correct_length_seq)

pad 1
pad 1
pad 1
pad 1
pad 1
nopad 0
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
nopad -261
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
pad 1
nopad -3871
pad 1
pad 1
pad 1
pad 1
pad 1


179928

In [79]:
print(len(df))
len(correct_length_seq)

179928


179928

In [81]:
correct_length_seq

[13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 9,
 7,
 7,
 7,
 9,
 9,
 9,
 9,
 7,
 7,
 7,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 

In [80]:
#df.loc[df['Id'] == '001-0071-00057',['Predicted']]=0
df['Predicted'] =correct_length_seq

,Id,Predicted
0,001-0071-00000,13
1,001-0071-00001,13
2,001-0071-00002,13
3,001-0071-00003,13
4,001-0071-00004,13
...,...,...
179923,003-0001-05902,13
179924,003-0001-05903,13
179925,003-0001-05904,13
179926,003-0001-05905,13


In [83]:
df[df['Id'].str.contains('001-0071-01')]

,Id,Predicted
1000,001-0071-01000,6
1001,001-0071-01001,6
1002,001-0071-01002,6
1003,001-0071-01003,6
1004,001-0071-01004,6
...,...,...
1317,001-0071-01317,13
1318,001-0071-01318,13
1319,001-0071-01319,13
1320,001-0071-01320,13


In [23]:
len(df[df['Id'].str.contains('001-0071-')])

1322

In [27]:
len('RALIHR_surgeon01_fps01_')

23

In [32]:
lenght

184003

In [33]:

for vid in vid2len:
    l1=vid2len[vid]
    lenght+=l1
    rows=vid[14:16]+'-'+vid[23:27]+'-'
    l2=len(df[df['Id'].str.contains(rows)])
    print(l2-l1)


1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
-261
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
-3871
1
1
1
1
1
